In [8]:
import sys
sys.path.append('../')

from logger import Logger

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
from selenium.webdriver.common.keys import Keys

import datetime
import json
import os


class IMDBLinkScraper:
    root_search_url = "https://www.imdb.com/search/title/?"

    # we will add this to the link with the formatting: ?release_date=2024-10-03,2024-10-03
    release_date_query = "&release_date="

    # the problem is that there are a lot of movies. for example there are 4,711 movies
    # in the day of 01-01-2023 and 2,719,230 movies in the year 2023 alone
    # so if we think about it, even if we only fetch the day 01-01-2023, there will be 4711 movies.
    # and IMDb made the search page in a way that it only shows 50 of the movies initially
    # and you have to click the "Show More" button to see 50 more movies. so we would need to click
    # 4711 / 50 times which is 95 clicks. lets say if every click and fetch takes 5 seconds. 95 x 5 = 471 seconds.
    # almost 8 minutes. for a day of movies. what? anyways we'll just test and see
    
    # what we will do is that we will first check every year individually and save the number of movies for that year
    # then we will calculate how should we adjust the release date parameter

    SCRAPING_MODES = ["DAY", "MONTH", "YEAR"]
    # if the year has > 500.000 movies, scraping mode will be DAY
    # if the year has > 50.000 movies, scraping mode will be MONTH
    # if the year has > 5.000 movies, scraping mode will be YEAR

    scraped_imdb_movie_links = []

    yearly_counts_file_path = "yearly_counts.json"
    
    def __init__(self):
        self.logger = Logger("imdb_link_scraper.log").getLogger()
        self.logger.info("starting IMDB Link scraper")

        self.driver = webdriver.Chrome()
        self.driver.wait = WebDriverWait(self.driver, 5)
        self.logger.info("initialized IMDb link scraper with " + self.driver.name + " driver.")

    def navigate(self, url):
        self.driver.get(url)
        self.logger.info("navigated to " + url)

    def create_search_query(self, start_date, end_date):
        return self.root_search_url + self.release_date_query + start_date + "," + end_date

    def read_yearly_movie_counts_file(self, filename):
        data_normalized = {}

        with open(filename, "r") as file:
            data = json.load(file)

            # we normalize the date while reading the file
            data_normalized["updated_at"] = datetime.datetime.strptime(data["updated_at"], "%Y-%m-%d")
            data_normalized["yearly_counts"] = {}

            for year in data["yearly_counts"]:
                data_normalized["yearly_counts"][year] = data["yearly_counts"][year]
        
        return data_normalized

    def write_yearly_movie_counts_file(self, filename, updated_at: any, yearly_counts):
        today = datetime.datetime.now().strftime("%Y-%m-%d")

        data_structured = {
            "updated_at": today,
            "yearly_counts": yearly_counts
        }

        with open(filename, "w") as file:
            self.logger.info("writing the yearly movie counts to JSON file: " + filename + " with indent=2. updated_at=" + updated_at)
            json.dump(data_structured, file, indent=2)

    # from 1800 to this day, fetch how many movies are there in every year
    # this is for understanding the data and how we should proceed fetching it
    def fetch_yearly_movie_count(self, start_year: int, end_year: int):
        # if a yearly_counts.json file exists and it is not older than 7 days, we just return the yearly_counts

        if (os.path.exists(self.yearly_counts_file_path)):
            self.logger.info("a yearly_counts.json file already exists. checking if it is viable")

            data = self.read_yearly_movie_counts_file(self.yearly_counts_file_path)

            updated_at = data["updated_at"] # this is already in datetime format
            now = datetime.datetime.now()
            seven_days_ago = now - datetime.timedelta(days=7)

            if updated_at > seven_days_ago:
                if str(start_year) in data["yearly_counts"] and str(end_year) in data["yearly_counts"]:
                    self.logger.info("yearly_counts.json file is viable. returning the data")
                    return data
                else:
                    self.logger.warning("yearly_counts.json file does not contain the needed years. going to start fetching new data")
            else:
                self.logger.info("yearly_counts.json file is older than 7 days. going to start fetching new data")
        else:
            self.logger.info("yearly_counts.json file does not exist. going to start fetching it")
            
                
        yearly_counts: dict = {}
        
        for year in range(start_year, end_year + 1):
            movie_count: int = 0
            
            start_date = str(year) + "-01-01"
            end_date = str(int(year + 1)) + "-01-01"

            self.logger.info("fetching movie count for interval: " + start_date + " | " + end_date)

            self.navigate(self.create_search_query(start_date, end_date))
            self.driver.implicitly_wait(10)

            movie_count_element_list = self.driver.find_elements(by=By.XPATH, value="//*[@id=\"__next\"]/main/div[2]/div[3]/section/section/div/section/section/div[2]/div/section/div[2]/div[2]/div[1]/div[1]")
            if (movie_count_element_list != []):
                movie_count_text: str = movie_count[0].text
                if movie_count_text == "":
                    movie_count = 0
                else:
                    movie_count = int(movie_count.split("of ")[1].replace(",", ""))
                
                print("movie count for year " + str(year) + ": " + str(movie_count))
                self.logger.info("found movie count for interval: " + start_date + " | " + end_date + " is " + str(movie_count))
            else:
                movie_count = 0
                print("movie count unknown for year " + str(year))
                self.logger.warning("movie count unknown for year " + str(year))

            yearly_counts[year] = movie_count

        self.logger.info("finished fetching movie counts for years " + str(start_year) + " to " + str(end_year))

        self.logger.info("writing the yearly movie counts to JSON file.")

        self.write_yearly_movie_counts_file(self.yearly_counts_file_path, yearly_counts)

        self.driver.quit()

        
    # TODO: do some refactoring

    # TODO: make a yearly movie count class inside this class
    
    # TODO: make a method that creates a scrape path. a scrape path will be a list of dates for the program to scrape while scraping the actual movies.
    # explanation: the yearly_counts.json file contains the yearly counts.
    # if the yearly count is > 500000, the scraping mode will be DAY.
    # and the program will create a series of days for scraping. like:
    # ["2024-01-01,2024-01-01", "2024-01-02,2024-01-02", ...]
    # if it was monthly, it wwould be like this:
    # ["2024-01-01,2024-01-31", "2024-02-01,2024-02-29"]
    
    


In [10]:
scraper = IMDBLinkScraper()
yearly_counts_data = scraper.fetch_yearly_movie_count(2000, 2024)

# turn yearly_counts into a json file
# add the todays date to the file itself
